In [1]:
import json
import pandas as pd
from pathlib import Path

# Get the path to the JSON file
file_path = Path.home() / "Downloads" / "jul17.json"

# Read the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

# Extract the "beta" data
beta_data = data["beta"]

# Create Series for each specified key
base_value = beta_data["base_value"]
holdings_series = pd.Series(beta_data["holdings"])
prices_series = pd.Series(beta_data["prices"])
min_W_series = pd.Series(beta_data["min_W"])
max_W_series = pd.Series(beta_data["max_W"])
new_target_weights_series = pd.Series(beta_data["new_target_weights"])
current_weights_series = pd.Series(beta_data["current_weights"])
external_movement = beta_data["external_movement"]

# Print the Series to verify
print("Holdings Series:")
print(holdings_series)
print("\nPrices Series:")
print(prices_series)
print("\nMin W Series:")
print(min_W_series)
print("\nMax W Series:")
print(max_W_series)
print("\nCurrent Weights Series:")
print(current_weights_series)

# If you need to convert any of these to DataFrames, you can do so like this:
# holdings_df = holdings_series.to_frame(name='holdings')

Holdings Series:
egld     1.599217e+01
form     3.704896e+04
xrp      2.985551e+02
perp     0.000000e+00
ldo      0.000000e+00
avax     1.662383e+01
enj      0.000000e+00
ftm      0.000000e+00
qnt      7.102291e+00
matic    4.160000e+02
looks    0.000000e+00
w        0.000000e+00
axl      6.941583e+02
mkr      0.000000e+00
ada      1.121564e+03
gala     0.000000e+00
beam     2.676564e+04
sushi    6.973561e+02
prime    5.821071e+01
rpl      3.126679e+01
cake     0.000000e+00
lqty     8.524390e-13
mana     1.569715e+03
blur     2.964534e+03
vet      2.281839e+03
oxt      0.000000e+00
crv      0.000000e+00
sol      3.305960e+00
cvx      0.000000e+00
audio    3.786238e+03
psp      0.000000e+00
aave     5.059195e+00
pyr      1.415019e+02
rly      0.000000e+00
myria    1.625509e+05
rari     2.607053e+02
imx      3.911726e+02
near     0.000000e+00
fil      1.397109e+02
grt      2.606874e+03
ens      1.842036e+01
flow     6.462422e+02
ygg      0.000000e+00
zrx      1.679973e+02
fet      3.7229

In [2]:
from portfolio_optimization.portfolio.rebalancing import *

optimize_trades(
  base_value=base_value,
  holdings=holdings_series,
  new_target_weights=new_target_weights_series,
  prices=prices_series,
  min_W=min_W_series,
  max_W=max_W_series,
  external_movement=external_movement
)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 17 09:30:04 PM: Your problem has 60 variables, 4 constraints, and 0 parameters.
(CVXPY) Jul 17 09:30:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 17 09:30:04 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 17 09:30:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 17 09:30:04 PM: Compiling problem (target solver=ECOS).
(CVXPY) Jul 17 09:30:04 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

egld    -6.522614e+00
form     3.363191e+06
xrp      4.928844e+02
perp     3.793639e+02
ldo      1.337042e+02
avax    -6.245797e+00
enj      1.433770e+03
ftm      5.001050e+02
qnt     -9.844340e-01
matic    1.720667e+02
looks    4.166335e+03
w        6.582250e+02
axl     -3.680625e+02
mkr      1.247720e-01
ada     -2.656393e+02
gala     1.022365e+04
beam    -1.304715e+04
sushi   -3.808282e+02
prime   -2.930915e+01
rpl     -1.487498e+01
cake     1.615780e+02
lqty     3.033138e+02
mana    -7.500224e+02
blur    -1.661950e+03
vet      8.768437e+03
oxt      3.813288e+03
crv      9.623692e+02
sol     -1.271113e+00
cvx      1.068539e+02
audio   -1.691090e+03
psp      1.854482e+04
aave    -1.950766e+00
pyr     -5.466058e+01
rly      4.540144e+04
myria   -8.182579e+04
rari    -8.759776e+01
imx     -2.239385e+02
near     4.537718e+01
fil     -7.730902e+01
grt     -1.443400e+03
ens     -1.074187e+01
flow    -1.672857e+02
ygg      4.492090e+02
zrx      6.088068e+02
fet     -2.073957e+02
arkm    -2